In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=my-postgres.cwzzbu8tu7mw.us-east-2.rds.amazonaws.com")
cur = conn.cursor()
conn.commit()

In [3]:
def get_data_files(path):
    file_array=[]
    for root, dirs, files in os.walk(path):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            file_array.append(os.path.abspath(f))
    return file_array

# Processing Song Data

In [4]:
song_files = get_data_files('data/song_data/')

In [5]:
path = song_files[0]
print(path)

C:\Users\Uvcan\Desktop\Data Modeling\data\song_data\A\A\A\TRAAAAW128F429D538.json


In [6]:
df = pd.read_json(path, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [7]:
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]

## Songs Table Data

In [8]:
# (song_id text, title text, artist_id int, year int, duration float)
song_data = [song_id, title, artist_id, year, duration]
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

## Artist Table Data

In [10]:
artist_data = [artist_id, artist_name, artist_location, artist_longitude, artist_latitude]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [12]:
conn.commit()
cur.execute(artist_table_insert, artist_data)
conn.commit()

# Process Log Data

In [14]:
log_files_data = get_data_files('data/log_data/')

In [15]:
filepath = log_files_data[0]
print(filepath)

C:\Users\Uvcan\Desktop\Data Modeling\data\log_data\2018\11\2018-11-01-events.json


In [16]:
df = pd.read_json(filepath, lines=True)
print(df.size)
df.head()

270


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## Time Table Data

In [ ]:
df = df[df['page']=='NextSong']
print(df.size)
df.head()

In [ ]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

In [ ]:
time_data = []
for line in t:
    time_data.append([line, line.hour, line.day, line.week, line.month, line.year, line.day_name()])
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [ ]:
time_data[1]

In [ ]:
# Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

### Insert Data Into Time Table

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

## User Table Data

In [ ]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

### Insert Records into User Table

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## Songs Play Table

#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`


#### Insert Records into Songplays Table
- We have implemented the `songplay_table_insert` query in sql_queries file, run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # (songplay_id int, start_time int, user_id int, level text, song_id text, artist_id text, session_id int, location text, user_agent text)
    # songplay_data = (index, row.ts, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    songplay_data = (index, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [ ]:
conn.close()